In [1]:
import cv2
import numpy as np
import glob

In [2]:
ref_image = cv2.imread('./newnew/Left_0850nm.png',0)

CHECKERBOARD = (6,7)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001) #maxiter = 30, epsilon = 0.0001

objpoints = []
imgpoints = [] 

objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None
# Defining checkerboard dimensions and size in real world coordinates (in mm)
objp = objp*4.235 + [[[4.235, 4.235, 0]]]
objp = objp.astype('float32')
objp

array([[[ 4.235   ,  4.235   ,  0.      ],
        [ 8.47    ,  4.235   ,  0.      ],
        [12.705   ,  4.235   ,  0.      ],
        [16.94    ,  4.235   ,  0.      ],
        [21.175001,  4.235   ,  0.      ],
        [25.410002,  4.235   ,  0.      ],
        [ 4.235   ,  8.47    ,  0.      ],
        [ 8.47    ,  8.47    ,  0.      ],
        [12.705   ,  8.47    ,  0.      ],
        [16.94    ,  8.47    ,  0.      ],
        [21.175001,  8.47    ,  0.      ],
        [25.410002,  8.47    ,  0.      ],
        [ 4.235   , 12.705   ,  0.      ],
        [ 8.47    , 12.705   ,  0.      ],
        [12.705   , 12.705   ,  0.      ],
        [16.94    , 12.705   ,  0.      ],
        [21.175001, 12.705   ,  0.      ],
        [25.410002, 12.705   ,  0.      ],
        [ 4.235   , 16.94    ,  0.      ],
        [ 8.47    , 16.94    ,  0.      ],
        [12.705   , 16.94    ,  0.      ],
        [16.94    , 16.94    ,  0.      ],
        [21.175001, 16.94    ,  0.      ],
        [25

In [3]:
images = glob.glob('./newnew/Left*850nm.png')
for fname in images:
    img = cv2.imread(fname, 0)
#     gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(img, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
    
    print(ret)
    """
    If desired number of corner are detected,
    we refine the pixel coordinates and display 
    them on the images of checker board
    """
    if ret == True:
        objpoints.append(objp)
        # refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(img, corners, (11,11),(-1,-1), criteria)
         
        imgpoints.append(corners2)
 
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
        
#         cv2.imshow('img', img)
#         cv2.waitKey(0)

#         cv2.destroyAllWindows()

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [4]:
        """
        Performing camera calibration by 
        passing the value of known 3D points (objpoints)
        and corresponding pixel coordinates of the 
        detected corners (imgpoints)
        """
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, ref_image.shape[::-1], None, None)

print(" ")
print("Reprojection error : \n")
print(ret)
print("Camera matrix : \n")
print(mtx)
print("dist : \n")
print(dist)
print("rvecs : \n")
print(rvecs)
print("tvecs : \n")
print(tvecs)

 
Reprojection error : 

0.3973604907454721
Camera matrix : 

[[2.24604014e+03 0.00000000e+00 7.52638170e+02]
 [0.00000000e+00 2.24403676e+03 6.26751195e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist : 

[[-2.20738272e-02  5.18569641e-01  7.50144724e-03  1.35536772e-03
  -2.89029305e+00]]
rvecs : 

(array([[ 0.1919524 ],
       [-0.20525309],
       [-1.58708205]]), array([[ 0.47454455],
       [-0.00631671],
       [-3.09175301]]), array([[ 4.79143921e-01],
       [ 2.29309893e-04],
       [-3.08750449e+00]]), array([[-0.27888444],
       [-0.15302431],
       [ 1.92322461]]), array([[-0.12627856],
       [-0.25119753],
       [ 1.05682182]]), array([[-0.12823174],
       [-0.27475671],
       [ 1.05923567]]), array([[-0.31807644],
       [-0.12822358],
       [ 2.25170244]]), array([[-0.31835864],
       [-0.11709472],
       [ 2.24123204]]), array([[-0.30162766],
       [-0.13586391],
       [ 2.11926985]]), array([[ 0.21309932],
       [-0.22059424],
       [-1.55366258

In [10]:
#undistortion
img = cv2.imread('./newnew/Left_14850nm.png')
h,  w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

# undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
dst = cv2.resize(dst, (0,0), fx = 0.5, fy = 0.5)

cv2.imshow('dst', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
#reprojection error
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )

total error: 0.06062111069831155
